# SPRINT - 3

## Item - Desenvolvimento do processamento dos dados para geração dos indicadores no Pipeline, e gravação na "Consumer Data"

## Item - Desenvolvimento do processamento dos dados para geração da consolidação de dados da produção no Pipeline, e gravação na "Consumer Data"

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [1]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta

import sys
sys.path.append('../../pods')

# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/sergiomedina/Downloads/edc-igti-smedina-4920e12ac565.json"

# Funções de integração com o Cloud Storage
from utilGCS import utilGCS
gcs = utilGCS(bucketName='edc-pa-i4-data')

# Funções diversas de manipulação de dados
from utilFuncs import utilFuncs
func = utilFuncs()


#### Dados na "processing-zone" pasta "DATAOP"

In [4]:
dtexec = "2021-11-12"
source = "dataop"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataop = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataop = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])

df_dataop['DTPROD'] = df_dataop['DTPROD'].astype(str)


In [3]:
df_dataop

#### Dados na "raw-data-zone" pasta "DATACONFIRM"

In [3]:
#dtexec = "2021-11-08"
source = "dataconfirm"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataconfirm = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df_dataconfirm = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec)])

df_dataconfirm['DTPROD'] = df_dataconfirm['DTPROD'].astype(str)

In [13]:
df_dataconfirm

,IDTURNO,DTAPONT,BATCH,PACKID,QTDUN,KGPACK,KGUNMED,DTPROD
0,1,2021-11-12 08:35:57,TB70407,1,50,289.750,5.795,2021-11-12
1,1,2021-11-12 09:37:19,TB70407,2,50,289.750,5.795,2021-11-12
2,1,2021-11-12 10:19:53,TB70407,3,50,289.750,5.795,2021-11-12
3,1,2021-11-12 11:17:24,TB70407,4,54,312.984,5.796,2021-11-12
4,1,2021-11-12 12:16:09,TB70407,5,54,312.984,5.796,2021-11-12
5,1,2021-11-12 13:17:32,TB70407,6,52,301.340,5.795,2021-11-12
6,2,2021-11-12 14:00:41,TB70407,7,54,312.984,5.796,2021-11-12
7,2,2021-11-12 15:03:28,TB70407,8,52,301.340,5.795,2021-11-12
8,2,2021-11-12 15:50:22,TB70407,9,49,284.004,5.796,2021-11-12
9,2,2021-11-12 16:41:51,TB70407,10,54,312.984,5.796,2021-11-12


In [4]:
df_dataconfirm.groupby(
    ['DTPROD', 'IDTURNO', 'BATCH']
).agg(
    {
        'PACKID': 'count',
        'QTDUN': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)


PACKID  QTDUN    KGPACK   KGUNMED
DTPROD     IDTURNO BATCH                                     
2021-11-12 1       TB70407       6    310  1796.558  5.795333
           2       TB70407       9    465  2694.832  5.795333
           3       TB70407       9    482  2793.458  5.795556

In [ ]:
df_dataconfirm.groupby(
    ['DTPROD', 'BATCH']
).agg(
    {
        'PACKID': 'count',
        'QTDUN': 'sum',
        'KGPACK': 'sum',
        'KGUNMED': 'mean'
    }
)

#### Dados na "processing-zone" pasta "DATAPROD"

In [5]:
#dtexec = "2021-11-08"
lineid = "101"
source = "dataprod"
folder = f"edc-pa-i4-data/processing-zone/{source}"

#df_dataprod = gcs.read_parquet_to_pandas(path=folder)
df_dataprod = gcs.read_parquet_to_pandas(path=folder, filters=[('DTPROD', '=', dtexec),('LINE', '=', lineid)])

df_dataprod['DTPROD'] = df_dataprod['DTPROD'].astype(str)
df_dataprod['LINE'] = df_dataprod['LINE'].astype(str)

In [ ]:
df_dataprod.to_csv(path_or_buf=f'/Users/ych885/Downloads/{dtexec}-df_dataprod.csv', sep=';', index=False )

In [14]:
#df_dataprod = df_dataprod.loc[(df_dataprod.DTPROD==dtexec) & (df_dataprod.LINE==lineid)]
df_dataprod

,IDTURNO,TIMESTAMP,OP,BATCH,TIMER,TOTMIN,STSID,STSDS,QTDPCS,QTDGOOD,QTDREJECT,DTPROD,LINE
0,1,2021-11-12 06:10:54,211112L101,TB70407,00:14:03,14.05,1,SETUP,14.0,0.0,14.0,2021-11-12,101
1,1,2021-11-12 06:24:44,211112L101,TB70407,00:13:50,13.83,1,SETUP,13.0,0.0,13.0,2021-11-12,101
2,1,2021-11-12 06:33:25,211112L101,TB70407,00:08:41,8.68,2,STARTUP,8.0,0.0,8.0,2021-11-12,101
3,1,2021-11-12 06:39:08,211112L101,TB70407,00:05:43,5.72,2,STARTUP,5.0,0.0,5.0,2021-11-12,101
4,1,2021-11-12 06:45:40,211112L101,TB70407,00:06:32,6.53,4,WORKING,6.0,6.0,0.0,2021-11-12,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,3,2021-11-13 05:35:00,211112L101,TB70407,00:13:58,13.97,0,STOP,0.0,0.0,0.0,2021-11-12,101
185,3,2021-11-13 05:35:14,211112L101,TB70407,00:00:14,0.23,0,STOP,0.0,0.0,0.0,2021-11-12,101
186,3,2021-11-13 05:39:20,211112L101,TB70407,00:04:06,4.10,0,STOP,0.0,0.0,0.0,2021-11-12,101
187,3,2021-11-13 05:42:10,211112L101,TB70407,00:02:50,2.83,0,STOP,0.0,0.0,0.0,2021-11-12,101


In [ ]:
df_dataprod.dtypes

In [ ]:
df_dataprod.where((df_dataprod.STSID>=4)).groupby(
    ['DTPROD','BATCH']
).agg(
    {
        'QTDPCS': 'sum',
        'QTDGOOD': 'sum',
        'QTDREJECT': 'sum'
    }
)

### Consolidando dados de Produção


In [6]:
df_prod = df_dataop.copy()

df_prod = df_prod.merge(
    pd.DataFrame(
        df_dataconfirm.groupby(
            ['DTPROD', 'BATCH']
        ).agg(
            {
                'PACKID': 'count',
                'QTDUN': 'sum',
                'KGPACK': 'sum',
                'KGUNMED': 'mean'
            }
        )        
    ).reset_index(), 
    how='inner',
    on=['DTPROD', 'BATCH']
)

df_prod = df_prod.merge( 
    pd.DataFrame(
        df_dataprod.where((df_dataprod.STSID>=4)).groupby(
            ['DTPROD','BATCH']
        ).agg(
            {
                'QTDPCS': 'sum',
                'QTDGOOD': 'sum',
                'QTDREJECT': 'sum',
                'TIMESTAMP': 'max'
            }
        )        
    ).reset_index(), 
    how='inner',
    on=['DTPROD', 'BATCH']
)

df_prod['TMAXLIMIT'] = df_prod.apply(lambda row:func.end_time(row['DTPROD'], 3),axis=1)
df_prod['TMAXLIMIT'] = pd.to_datetime(df_prod['TMAXLIMIT'])
df_prod['TMAXDIFF'] = (df_prod['TMAXLIMIT']-df_prod['TIMESTAMP']).dt.seconds/60
df_prod['QTDWIP'] = df_prod.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df_prod.rename(columns = {'PACKID':'QTDPACKS', 'QTDUN':'QTDCONFIRM', 'KGPACK':'TOTKGPACK'}, inplace = True)

df_prod['QTDIFF'] = df_prod['QTDCONFIRM'] - df_prod['QTDGOOD']

df_prod = df_prod[['DTPROD', 'OP', 'DTINI', 'DTFIM', 'CODMAT', 'BATCH', 'QTDPLAN', 'QTDCONFIRM', 'QTDIFF', 'QTDPCS', 'QTDGOOD', 'QTDREJECT', 'QTDWIP', 'QTDPACKS', 'TOTKGPACK', 'KGUNMED']]

df_prod

,DTPROD,OP,DTINI,DTFIM,CODMAT,BATCH,QTDPLAN,QTDCONFIRM,QTDIFF,QTDPCS,QTDGOOD,QTDREJECT,QTDWIP,QTDPACKS,TOTKGPACK,KGUNMED
0,2021-11-12,211112L101,2021-11-12 06:00:00,2021-11-13 05:59:59,TB70PVC,TB70407,1320,1257,0.0,1270.0,1257.0,13.0,38.0,24,7284.848,5.795417


In [ ]:
df_prod.to_csv(path_or_buf=f'/Users/ych885/Downloads/{dtexec}-df_prod.csv', sep=';', index=False, decimal=',' )

In [ ]:
(df_prod['QTDCONFIRM']/df_prod['QTDPLAN'])

#### Dados na "processing-zone" pasta "DATAPROD" - CALC OEE

In [17]:
# Totalizando por Data, turno, Linha e Status
df_sts = pd.DataFrame(df_dataprod.groupby(
#    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH', 'STSID', 'STSDS']
    ['DTPROD', 'IDTURNO', 'LINE', 'STSID', 'STSDS']
).agg(
    {
        'TOTMIN': 'sum',
        'QTDPCS': 'sum',
        'QTDGOOD': 'sum',
        'QTDREJECT': 'sum'
    }
)).reset_index()


# Clonando a base carregada dno DATAPROD
df = df_dataprod.copy()

# Preparando para consolidação de dados por Data, turno e Linha 
df['TMIN'] = df['TIMESTAMP']
df['TMAX'] = df['TIMESTAMP']
df.drop(['TIMESTAMP'], axis=1, inplace=True)

df = pd.DataFrame(
    df.where((df.STSID>=4)).groupby(
        # ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
        ['DTPROD', 'IDTURNO', 'LINE']
    ).agg(
        {
            'TOTMIN': 'sum',
            'QTDPCS': 'sum',
            'QTDGOOD': 'sum',
            'QTDREJECT': 'sum',
            'TMIN': 'min',
            'TMAX': 'max'
        }
    )
).reset_index()


# Tratamento das informações
df['TMAXLIMIT'] = df.apply(lambda row:func.end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMAXLIMIT'] = pd.to_datetime(df['TMAXLIMIT'])
df['TMAXDIFF'] = (df['TMAXLIMIT']-df['TMAX']).dt.seconds/60
df['QTDWIP'] = df.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df['TMINLIMIT'] = df.apply(lambda row:func.start_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMINLIMIT'] = pd.to_datetime(df['TMINLIMIT'])
df['TMINDIFF'] = (df['TMIN']-df['TMINLIMIT']).dt.seconds/60

df['DIFF'] = df['TMAXDIFF'].diff().fillna(0)

df['TOTMINADJUSTED'] = round(df['TOTMIN'] + df['DIFF'],0)

df.drop(['TMAX', 'TMAXLIMIT', 'TMAXDIFF'], axis=1, inplace=True)
df.drop(['TMIN', 'TMINLIMIT', 'TMINDIFF', 'DIFF'], axis=1, inplace=True)

# Tot. Tempo turno
df['TPTOTAL'] = 480

print('DEBUG -> [ANTES AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

df['TOTMINADJUSTED'] = df.apply(lambda row:(row['TOTMINADJUSTED'] if row['TOTMINADJUSTED']<480 else 480),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TOTMINADJUSTED=',df['TOTMINADJUSTED'].sum(),'\n\r',df['TOTMINADJUSTED'])

# Tot. Tempo = WORKING
df['TPWORKING'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=4) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

print('DEBUG -> [ANTES AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

df['TPWORKING'] = df.apply(lambda row:(row['TPWORKING'] if row['TPWORKING']<=row['TOTMINADJUSTED'] else row['TOTMINADJUSTED']),axis=1)

print('DEBUG -> [DEPOIS AJUSTE]:','TPWORKING=',df['TPWORKING'].sum(),'\n\r',df['TPWORKING'])

# Tot. Tempo = TPSTOPPLAN
df['TPSTOPPLAN'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

# Tot. Tempo = TPNOALLOC
df['TPNOALLOC'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==0) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)    


# Tempo Programado para produzir
df['TPPROG'] = df['TPTOTAL'] - (df['TPNOALLOC']+df['TPSTOPPLAN'])


# Temp Ociosidade
df['TPIDLE'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=1) &
    (df_sts.STSID<3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) 
    # &
    # (df_sts.OP==row['OP']) &
    # (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPIDLE'] = ((df['TPPROG']-(df['TPWORKING'] + df['TPIDLE']))+df['TPIDLE'])

# Unidades - Produção Teórica
df['QTDPCSTHEOR'] = df['TPWORKING'] * 1

# Unidades - Perda por performance
df['QTDPCSLOSS'] = df['QTDPCSTHEOR'] - df['QTDPCS']

# OEE Calc 
df['OEEDISP'] = round(df['TPWORKING'] / df['TPPROG'],3)
df['OEEPERF'] = round(df['QTDPCS'] / df['QTDPCSTHEOR'],3)
df['OEEQUAL'] = round(df['QTDGOOD'] / df['QTDPCS'],3)
df['OEE'] = round(df['OEEDISP'] * df['OEEPERF'] * df['OEEQUAL'],3)

DEBUG -> [ANTES AJUSTE]: TOTMINADJUSTED= 1399.0 
 0    439.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TOTMINADJUSTED= 1399.0 
 0    439.0
1    480.0
2    480.0
Name: TOTMINADJUSTED, dtype: float64
DEBUG -> [ANTES AJUSTE]: TPWORKING= 1362.0 
 0    438.64
1    481.59
2    441.77
Name: TPWORKING, dtype: float64
DEBUG -> [DEPOIS AJUSTE]: TPWORKING= 1360.4099999999999 
 0    438.64
1    480.00
2    441.77
Name: TPWORKING, dtype: float64


In [15]:
df.dtypes

DTPROD             object
IDTURNO           float64
LINE               object
TOTMIN            float64
QTDPCS            float64
QTDGOOD           float64
QTDREJECT         float64
QTDWIP            float64
TOTMINADJUSTED    float64
TPTOTAL             int64
TPWORKING         float64
TPSTOPPLAN        float64
TPNOALLOC         float64
TPPROG            float64
TPIDLE            float64
QTDPCSTHEOR       float64
QTDPCSLOSS        float64
OEEDISP           float64
OEEPERF           float64
OEEQUAL           float64
OEE               float64
dtype: object

In [19]:
df[['DTPROD','IDTURNO','LINE','TPTOTAL','TPPROG','TPWORKING','TOTMINADJUSTED','TPSTOPPLAN','TPNOALLOC','TPIDLE']]


# TOTMIN            float64


,DTPROD,IDTURNO,LINE,TPTOTAL,TPPROG,TPWORKING,TOTMINADJUSTED,TPSTOPPLAN,TPNOALLOC,TPIDLE
0,2021-11-12,1.0,101,480,480.00,438.64,439.0,0.0,0.00,41.36
1,2021-11-12,2.0,101,480,480.00,480.00,480.0,0.0,0.00,0.00
2,2021-11-12,3.0,101,480,452.54,441.77,480.0,0.0,27.46,10.77


In [21]:
df[['DTPROD','IDTURNO','QTDPCS','QTDGOOD','QTDREJECT','QTDWIP','QTDPCSTHEOR','QTDPCSLOSS']]

,DTPROD,IDTURNO,QTDPCS,QTDGOOD,QTDREJECT,QTDWIP,QTDPCSTHEOR,QTDPCSLOSS
0,2021-11-12,1.0,408.0,405.0,3.0,2.0,438.64,30.64
1,2021-11-12,2.0,456.0,451.0,5.0,0.0,480.00,24.00
2,2021-11-12,3.0,406.0,401.0,5.0,38.0,441.77,35.77


In [22]:
df[['DTPROD','IDTURNO','LINE','OEEDISP', 'OEEPERF', 'OEEQUAL', 'OEE']]

,DTPROD,IDTURNO,LINE,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-12,1.0,101,0.914,0.930,0.993,0.844
1,2021-11-12,2.0,101,1.000,0.950,0.989,0.940
2,2021-11-12,3.0,101,0.976,0.919,0.988,0.886


In [11]:

df_oee = pd.DataFrame(
    df.groupby(
        ['DTPROD']
    ).agg(
        {
            'TPTOTAL': 'sum',
            'TPPROG': 'sum',
            'TPWORKING': 'sum',
            'TPSTOPPLAN': 'sum',
            'TPNOALLOC': 'sum',
            'TPIDLE': 'sum',
            'QTDPCS': 'sum',
            'QTDGOOD': 'sum',
            'QTDREJECT': 'sum',
            'QTDPCSTHEOR': 'sum',
            'QTDPCSLOSS': 'sum',

        }
    )
).reset_index()

df_oee['OEEDISP'] = round(df_oee['TPWORKING']/df_oee['TPPROG'],3)

df_oee['OEEPERF'] = round(df_oee['QTDPCS'] / df_oee['QTDPCSTHEOR'],3)

df_oee['OEEQUAL'] = round(df_oee['QTDGOOD'] / df_oee['QTDPCS'],3)

df_oee['OEE'] = round(df_oee['OEEDISP'] * df_oee['OEEPERF'] * df_oee['OEEQUAL'],3)


df_oee


,DTPROD,TPTOTAL,TPPROG,TPWORKING,TPSTOPPLAN,TPNOALLOC,TPIDLE,QTDPCS,QTDGOOD,QTDREJECT,QTDPCSTHEOR,QTDPCSLOSS,OEEDISP,OEEPERF,OEEQUAL,OEE
0,2021-11-12,1440,1412.54,1360.41,0.0,27.46,52.13,1270.0,1257.0,13.0,1360.41,90.41,0.963,0.934,0.99,0.89


In [ ]:
# df.agg(
#     {
#         'OEEDISP': 'mean',
#         'OEEPERF': 'mean',
#         'OEEQUAL': 'mean',
#         'OEE': 'mean'
#     }
# )


df.groupby(
    ['DTPROD']
).agg(
    {
        'OEEDISP': 'mean',
        'OEEPERF': 'mean',
        'OEEQUAL': 'mean',
        'OEE': 'mean'
    }
)


